# *Projet Python ENSAE*

## Tokenisation d'actifs immobiliers - external data

Ce notebook a pour objectif de **créer un fichier .csv des données externes** issues de BDD INSEE que nous utilisons dans notre modèle de valorisation des actifs immobiliers.

# Etape 0 : packages

In [1]:
# importation des packages importants

import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.neighbors import BallTree
import requests
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
!pip install nltk
from nltk.metrics.distance import jaro_winkler_similarity
from scipy.spatial.distance import cdist
!pip install unidecode
import unidecode
import zipfile
from urllib.request import urlopen
import shutil
import os
!pip install openpyxl

# Etape 1 : importation des bases de données INSEE

### 1.1. Dictionnaire départements

In [2]:
DEPARTMENTS = {
    '01': 'Ain', 
    '02': 'Aisne', 
    '03': 'Allier', 
    '04': 'Alpes-de-Haute-Provence', 
    '05': 'Hautes-Alpes',
    '06': 'Alpes-Maritimes', 
    '07': 'Ardèche', 
    '08': 'Ardennes', 
    '09': 'Ariège', 
    '10': 'Aube', 
    '11': 'Aude',
    '12': 'Aveyron', 
    '13': 'Bouches-du-Rhône', 
    '14': 'Calvados', 
    '15': 'Cantal', 
    '16': 'Charente',
    '17': 'Charente-Maritime', 
    '18': 'Cher', 
    '19': 'Corrèze', 
    '2A': 'Corse-du-Sud', 
    '2B': 'Haute-Corse',
    '21': 'Côte-d\'Or', 
    '22': 'Côtes-d\'Armor', 
    '23': 'Creuse', 
    '24': 'Dordogne', 
    '25': 'Doubs', 
    '26': 'Drôme',
    '27': 'Eure', 
    '28': 'Eure-et-Loir', 
    '29': 'Finistère', 
    '30': 'Gard', 
    '31': 'Haute-Garonne', 
    '32': 'Gers',
    '33': 'Gironde', 
    '34': 'Hérault', 
    '35': 'Ille-et-Vilaine', 
    '36': 'Indre', 
    '37': 'Indre-et-Loire',
    '38': 'Isère', 
    '39': 'Jura', 
    '40': 'Landes', 
    '41': 'Loir-et-Cher', 
    '42': 'Loire', 
    '43': 'Haute-Loire',
    '44': 'Loire-Atlantique', 
    '45': 'Loiret', 
    '46': 'Lot', 
    '47': 'Lot-et-Garonne', 
    '48': 'Lozère',
    '49': 'Maine-et-Loire', 
    '50': 'Manche', 
    '51': 'Marne', 
    '52': 'Haute-Marne', 
    '53': 'Mayenne',
    '54': 'Meurthe-et-Moselle', 
    '55': 'Meuse', 
    '56': 'Morbihan', 
    '57': 'Moselle', 
    '58': 'Nièvre', 
    '59': 'Nord',
    '60': 'Oise', 
    '61': 'Orne', 
    '62': 'Pas-de-Calais', 
    '63': 'Puy-de-Dôme', 
    '64': 'Pyrénées-Atlantiques',
    '65': 'Hautes-Pyrénées', 
    '66': 'Pyrénées-Orientales', 
    '67': 'Bas-Rhin', 
    '68': 'Haut-Rhin', 
    '69': 'Rhône',
    '70': 'Haute-Saône', 
    '71': 'Saône-et-Loire', 
    '72': 'Sarthe', 
    '73': 'Savoie', 
    '74': 'Haute-Savoie',
    '75': 'Paris', 
    '76': 'Seine-Maritime', 
    '77': 'Seine-et-Marne', 
    '78': 'Yvelines', 
    '79': 'Deux-Sèvres',
    '80': 'Somme', 
    '81': 'Tarn', 
    '82': 'Tarn-et-Garonne', 
    '83': 'Var', 
    '84': 'Vaucluse', 
    '85': 'Vendée',
    '86': 'Vienne', 
    '87': 'Haute-Vienne', 
    '88': 'Vosges', 
    '89': 'Yonne', 
    '90': 'Territoire de Belfort',
    '91': 'Essonne', 
    '92': 'Hauts-de-Seine', 
    '93': 'Seine-Saint-Denis', 
    '94': 'Val-de-Marne', 
    '95': 'Val-d\'Oise',
    '971': 'Guadeloupe', 
    '972': 'Martinique', 
    '973': 'Guyane', 
    '974': 'La Réunion',
    '975': 'Saint-Pierre et Miquelon',
    '976': 'Mayotte',
}

flipped_dict = dict(zip(DEPARTMENTS.values(), DEPARTMENTS.keys()))

### 1.2. Taux de natalité :

Lien fichiers naissance par département : https://www.insee.fr/fr/statistiques/2540004?sommaire=4767262.

Nous retenons les fichiers par départements de naissance (csv, 12 mo).

In [3]:
# Importation des données :

link = "https://www.insee.fr/fr/statistiques/fichier/2012761/TCRD_053.xlsx"

df_natalite = pd.read_excel(link,"DEP", header = 3, skipfooter = 2, 
                                 names = np.array(["code_dep", "nom_dep", "tx_natalite_2020_percent"]),
                                usecols = "A,B,C")

In [4]:
# Visulaisation des données :

df_natalite.sample(3)

,code_dep,nom_dep,tx_natalite_2020_percent
30,30,Gard,9.8
37,37,Indre-et-Loire,9.9
68,68,Haut-Rhin,9.9


### 1.3. Musées :

Lien liste et localisation des musées en France : https://www.data.gouv.fr/fr/datasets/liste-et-localisation-des-musees-de-france/.

Nous retenons la liste des musées de France au 31/12/2017 (238.3 ko).

In [5]:
musees = pd.read_excel("https://www.data.gouv.fr/fr/datasets/r/22df4a13-72d8-4b34-940e-8aec297b5ded",
                      usecols = "A,B,E:I")

In [6]:
musees.head(3)

,NEW REGIONS,NOMDEP,ID MUSEE,NOM DU MUSEE,ADR,CP,VILLE
0,AUVERGNE-RHÔNE-ALPES,AIN,0105302,Musée Départemental des Pays De l'Ain,"34, rue du Général Delestraint",1000.0,BOURG-EN-BRESSE
1,AUVERGNE-RHÔNE-ALPES,AIN,0105301,Musée du Brou,"Monastère Royal de Brou\n63, Boulevard de Brou",1000.0,BOURG-EN-BRESSE
2,AUVERGNE-RHÔNE-ALPES,AIN,0106401,Musée de la Société d'Histoire et d'Archéologie,Mairie,1470.0,BRIORD


In [7]:
df_musees = musees.groupby("NOMDEP").size().reset_index()

df_musees = df_musees.rename(columns = {"NOMDEP":"nom_dep", 0:"nb_musees"})

In [8]:
# Création d'un array à partir des clés du dictionnaire qui donne nom_departement --> numero
arr_dic = np.array([unidecode.unidecode(word.lower()) for word in flipped_dict.keys()])

# Recréation du dictionnaire avec des clés dans le même format que pour l'array au dessus
dic_dpt_lower = {}

for key, values in flipped_dict.items():
    dic_dpt_lower.update({unidecode.unidecode(key.lower()):values})

In [9]:
# Création d'un dictionnaire à partir des noms des départements dans notre dataframe sur les musées
arr_dpt_mus = np.array(df_musees["nom_dep"].str.lower())

In [10]:
# Création d'une matrice avec pour chaque case la similarité entre le nom du département venant du df musées (en ligne)
# et celui venant de notre dictionnaire (en colonnes, donc arr_dic est en colonnes)

matrix = cdist(arr_dpt_mus.reshape(-1, 1), arr_dic.reshape(-1, 1), lambda x, y: jaro_winkler_similarity(x[0], y[0]))

# Puisqu'on a un score de similarité, on garde pour chaque ligne l'index de la valeur la plus élevée
correct = np.argmax(matrix, axis = 1)

# Cette boucle permet de vérifier que les associations sont correctes :

#for i in range(len(correct)):
    #print(arr_dic[correct[i]], arr_dpt_mus[i])
    #print("---")


'\nfor i in range(len(correct)):\n    print(arr_dic[correct[i]], arr_dpt_mus[i])\n    print("---")\n'

In [11]:
# Ajout de la colonne qui nous intéresse dans df_musees
df_musees["code_dep"] = " "

for i in range(df_musees.shape[0]):
    """
    Peu élégant, mais fonctionnel, il faudrait réfléchir un peu pour utiliser amap ou apply...
    """
    # Les deux variables ci-dessous ont pour seule utilité de rendre le code un tout petit peu plus lisible
    val_inter = list(arr_dpt_mus).index(df_musees["nom_dep"][i].lower())
    code_dep = dic_dpt_lower[arr_dic[correct[val_inter]]]
    df_musees["code_dep"][i] = code_dep

C:\Users\leo\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [12]:
display(df_musees)

,nom_dep,nb_musees,code_dep
0,AIN,14,01
1,AISNE,15,02
2,ALLIER,9,03
3,ALPES DE HAUTE PROVENCE,9,04
4,ALPES-MARITIMES,33,06
...,...,...,...
96,VENDEE,14,85
97,VIENNE,12,86
98,VOSGES,7,88
99,YONNE,12,89


### 1.4. Taux de chômage :

In [13]:
# importation des données 

url = "https://www.insee.fr/fr/statistiques/series/csv/famille/102760732"
file_name = 'famille_TAUX-CHOMAGE_23122021.zip'

# extracting zipfile from URL
with urlopen(url) as response, open(file_name, 'wb') as out_file:
   shutil.copyfileobj(response, out_file)

# extracting required file from zipfile
with zipfile.ZipFile(file_name) as zf:
   zf.extract('valeurs_trimestrielles.csv')

# deleting the zipfile from the directory
os.remove('famille_TAUX-CHOMAGE_23122021.zip')

# loading data from the file
chomage = pd.read_csv('valeurs_trimestrielles.csv', sep=";")

chomage = chomage.rename(columns = {"Libellé": "Libelle"})

# on fait un peu de nettoyage
chomage_vf = chomage.drop(chomage[chomage["Libelle"] == "Codes"].index)
chomage_vf = chomage_vf.reset_index()
chomage_vf.shape

chomage_vf["Libelle"] = [nom.split(" ")[7] for nom in chomage_vf["Libelle"]]
chomage_vf = chomage_vf.drop(columns = ["index", "idBank", "Dernière mise à jour", "Période"])

In [14]:
# On ne garde que les valeurs depuis 2016
liste_cols = ["Libelle"]
annees = ["20"+str(i) for i in range(16,21)]
trimestres = ["-T"+str(i) for i in range(1,5)]

for an in annees:
    for tr in trimestres:
        liste_cols.append(an+tr)
liste_cols.append("2021-T1")
liste_cols.append("2021-T2")

chomage_vf = chomage_vf.filter(liste_cols, axis=1)

chomage_vf = chomage_vf.replace(to_replace = 'La', value = 'La Réunion', regex=True)
chomage_vf = chomage_vf.replace(to_replace = 'Territoire', value = 'Territoire de Belfort', regex=True)
chomage_vf.isna().sum().any() # Pas de NA dans le df a priori

False

In [15]:
liste_cols = liste_cols[1:]

# Les valeurs étaient des objets, on les transforme en entiers
chomage_vf = chomage_vf.apply(lambda x: x.astype(float) if x.name in liste_cols else x)

for an in annees:
    chomage_vf[an] = chomage_vf[[an+str(tr) for tr in trimestres]].mean(axis = 1)

chomage_vf['2021'] = chomage_vf[["2021-T1","2021-T2"]].mean(axis=1)
display(chomage_vf)

,Libelle,2016-T1,2016-T2,2016-T3,2016-T4,2017-T1,2017-T2,2017-T3,2017-T4,2018-T1,...,2020-T3,2020-T4,2021-T1,2021-T2,2016,2017,2018,2019,2020,2021
0,France,9.9,9.8,9.6,9.7,9.3,9.2,9.3,8.7,8.9,...,8.9,7.8,7.8,7.8,9.750,9.125,8.700,8.175,7.850,7.80
1,Île-de-France,8.8,8.7,8.6,8.8,8.3,8.2,8.3,7.9,8.1,...,8.4,7.7,7.7,7.6,8.725,8.175,7.925,7.400,7.350,7.65
2,Centre-Val,9.7,9.5,9.4,9.4,9.0,9.0,8.9,8.4,8.6,...,8.3,7.1,7.2,7.2,9.500,8.825,8.450,8.050,7.400,7.20
3,Pays,8.8,8.5,8.4,8.4,8.0,7.9,7.9,7.3,7.6,...,8.0,6.7,6.8,6.7,8.525,7.775,7.375,7.050,7.000,6.75
4,Bretagne,8.8,8.6,8.4,8.6,8.1,8.0,8.0,7.5,7.7,...,7.7,6.6,6.6,6.5,8.600,7.900,7.500,6.975,6.775,6.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,Var,11.1,10.8,10.7,11.0,10.6,10.4,10.4,9.9,10.2,...,9.7,8.2,8.4,8.6,10.900,10.325,9.925,9.200,8.575,8.50
110,Guadeloupe,25.6,25.1,21.5,20.6,21.6,22.9,21.4,22.5,23.5,...,18.9,19.4,17.5,19.2,23.200,22.100,22.025,20.125,16.850,18.35
111,Martinique,19.8,19.7,19.1,17.4,19.4,19.2,18.3,17.3,16.5,...,16.4,15.0,12.7,14.7,19.000,18.550,18.475,16.150,13.050,13.70
112,Guyane,19.4,19.8,22.8,19.1,18.7,20.4,20.5,18.1,15.8,...,17.1,17.8,12.5,13.8,20.275,19.425,16.825,17.125,14.125,13.15


In [16]:
chomage_vf["Département"] = chomage_vf["Libelle"].map(flipped_dict)
chomage_vf.drop(chomage_vf[chomage_vf['Département'].isnull()].index, inplace=True)

chomage_final = chomage_vf[["Département","Libelle","2016","2017","2018","2019","2020","2021"]].copy()
chomage_final = chomage_final.rename(columns = {"Département": "code_dep", "Libelle": "nom_dep", "2016": "chomage_2016",
                               "2017": "chomage_2017", "2018": "chomage_2018", "2019": "chomage_2019",
                               "2020": "chomage_2020", "2021": "chomage_2019"})


chomage_final = chomage_final.reset_index().drop("index", axis =1).sort_values(by = 'code_dep')
display(chomage_final.head(2))

,code_dep,nom_dep,chomage_2016,chomage_2017,chomage_2018,chomage_2019,chomage_2020,chomage_2019
1,01,Ain,7.35,6.775,6.325,6.050,6.075,6.05
2,02,Aisne,13.80,13.175,12.575,11.775,11.125,11.15


### 1.5. Taux de logements vacants :

En théorie, plus le taux de logements vacants est élevé dans une zone, plus les prix sont bas.

In [17]:
link = 'https://www.data.gouv.fr/fr/datasets/r/d6b298be-42ee-4a9f-9a01-c8cb6edd4978'
vacants = pd.read_excel(link, 'Données')

In [18]:
vacants_new = vacants[["INSEE_COM","CODE_DEPT","NOM_DEPT","Prop_logvac_pp_010119"]]
vacants_new.head()
vacants_new = vacants_new.rename(columns = {"INSEE_COM": "code_postal", "CODE_DEPT": "code_dep", "NOM_DEPT": "nom_dep",
                                           "Prop_logvac_pp_010119": "taux_vacances_2019"})


vacants_new = vacants_new.groupby("nom_dep").mean("taux_vacance_2019").reset_index()
vacants_new["code_dep"] = vacants_new["nom_dep"].map(flipped_dict)
vacants_new = vacants_new.sort_values(by = "code_dep").reset_index().drop(columns = "index")
vacants_new

,nom_dep,taux_vacances_2019,code_dep
0,Ain,10.066116,01
1,Aisne,10.998363,02
2,Allier,14.294594,03
3,Alpes-de-Haute-Provence,11.413979,04
4,Hautes-Alpes,9.437147,05
...,...,...,...
95,Val-d'Oise,8.253955,95
96,Guadeloupe,16.804188,971
97,Martinique,13.315198,972
98,Guyane,15.635261,973


# Etape 2 : jointure des tables

In [19]:
dep_nata = [i for i in df_natalite["code_dep"]]
dep_musees = [j for j in df_musees["code_dep"]]
dep_chomage = [j for j in chomage_final["code_dep"]]
dep_vac = [j for j in vacants_new["code_dep"]]

# On fait l'intersection successive des départements de chaque df
dep_commun = list(set(dep_nata).intersection(set(dep_musees).intersection(set(dep_chomage).intersection(dep_vac))))
display(len(dep_commun))

99

**Bonne nouvelle : 99 départements en commun et autant dans notre df d'origine !**

In [20]:
not_commun_nata = [i for i in dep_nata if i not in dep_commun]
# display(not_commun_nata)

not_commun_musees = [i for i in dep_musees if i not in dep_commun]
# display(not_commun_musees)

not_commun_chomage = [i for i in dep_chomage if i not in dep_commun]
# display(not_commun_chomage)

not_commun_vac = [i for i in dep_vac if i not in dep_commun]
# display(not_commun_vac)

Les Landes (**40**) sont absentes du df chomage.
Saint-Pierre et Miquelon (**975**) et Mayotte (**976**) ne sont pas présents partout.

La nature du **M** n'est pas encore identifiée...

In [21]:
df_musees.set_index("code_dep")
print()

In [22]:
df_commun = (df_natalite.set_index("code_dep")
            .join(df_musees.set_index("code_dep"),     how = "inner", rsuffix = "jo")
            .join(chomage_final.set_index("code_dep"), how = "inner", rsuffix = "jo1")
            .join(vacants_new.set_index("code_dep"),   how = "inner", rsuffix = "jo2"))
df_commun.sample(5)

,nom_dep,tx_natalite_2020_percent,nom_depjo,nb_musees,nom_depjo1,chomage_2016,chomage_2017,chomage_2018,chomage_2019,chomage_2020,chomage_2019,nom_depjo2,taux_vacances_2019
code_dep,,,,,,,,,,,,,
03,Allier,7.8,ALLIER,9,Allier,10.525,9.900,9.525,9.050,8.650,8.45,Allier,14.294594
94,Val-de-Marne,13.8,VAL DE MARNE,8,Val-de-Marne,8.825,8.375,8.050,7.475,7.475,7.80,Val-de-Marne,7.359182
79,Deux-Sèvres,8.4,DEUX-SEVRES,8,Deux-Sèvres,7.825,7.125,6.700,6.100,5.925,5.65,Deux-Sèvres,9.419295
52,Haute-Marne,8.2,HAUTE-MARNE,7,Haute-Marne,9.250,8.350,7.900,7.225,6.950,6.55,Haute-Marne,13.974250
12,Aveyron,7.8,AVEYRON,11,Aveyron,7.300,6.900,6.575,6.275,6.025,6.00,Aveyron,13.496663


En comparant les colonnes nom_dep, nom_depjo1 & nom_depjo2, on constate que **la jointure semble a fonctionné correctement**.

In [23]:
df_commun = df_commun.drop(columns = ["nom_depjo", "nom_depjo1", "nom_depjo2"]).reset_index()
print(f"Il y a {len(df_commun['nom_dep'].unique())} départements représentés.")
df_commun.head(4)

Il y a 99 départements représentés.


,code_dep,nom_dep,tx_natalite_2020_percent,nb_musees,chomage_2016,chomage_2017,chomage_2018,chomage_2019,chomage_2020,chomage_2019,taux_vacances_2019
0,01,Ain,10.3,14,7.350,6.775,6.325,6.050,6.075,6.05,10.066116
1,02,Aisne,10.1,15,13.800,13.175,12.575,11.775,11.125,11.15,10.998363
2,03,Allier,7.8,9,10.525,9.900,9.525,9.050,8.650,8.45,14.294594
3,04,Alpes-de-Haute-Provence,8.0,9,11.450,11.075,10.750,10.075,9.400,9.35,11.413979


**On a bien les 99 lignes correspondant aux départements**. On exporte désormais ces données au format csv pour pouvoir les importer ensuite dans le notebook principal ("valo-immo").

In [27]:
# Ancienne version gardée pour la forme où l'on passe par des fichiers .csv 
#Très peu propre mais nous manquions de temps pour mettre en place autre chose
#df_commun.to_csv('external-data.csv')

(99, 11)

In [25]:
df_commun.shape
def retrieve_data():
    """
    Très peu propre mais fonctionnel
    """
    return df_commun